In [2]:
import boto3
import sagemaker
from sagemaker.sklearn.estimator import SKLearn

In [3]:
type(SKLearn)

abc.ABCMeta

In [4]:
import pandas as pd

In [5]:
RANDOM_STATE=101
SMALL_DATASET=False

BUCKET = "datascience-hbo-users"

In [6]:
region = boto3.Session().region_name
smclient = boto3.Session().client('sagemaker')
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.Session(default_bucket=BUCKET)

Couldn't call 'get_role' to get Role ARN from role name sagemaker-run-notebook-ExecuteNotebookClientRole-1Q9SQ10V8KNNK to get Role path.


In [7]:
sklearn_preprocessor = SKLearn(
    entry_point='featurizer.py',
    role=role,
    train_instance_type="ml.c4.xlarge",
    sagemaker_session=sagemaker_session)

This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.


In [8]:
type(sklearn_preprocessor)

sagemaker.sklearn.estimator.SKLearn

In [ ]:
train_input = 's3://datascience-hbo-users/users/rc/FT_propensity/7_day/snowflake-hbomax-staging/raw_data.csv_0_0_0.csv'

In [ ]:
sklearn_preprocessor.fit({'train': train_input})

In [ ]:
sklearn_preprocessor.model_data

In [ ]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transformer = sklearn_preprocessor.transformer(
    instance_count=1, 
    instance_type='ml.m4.xlarge',
    assemble_with = 'Line',
    accept = 'text/csv')

In [ ]:
test_input = 's3://datascience-hbo-users/sagemaker-scikit-learn-2020-07-23-14-52-26-706/test_input.csv'

In [ ]:
# Preprocess training input
transformer.transform(test_input, content_type='text/csv', split_type='Line')
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()
preprocessed_train = transformer.output_path
preprocessed_train

In [ ]:
df_test = pd.read_csv(f'{preprocessed_train}/test_input.csv.out')

In [ ]:
df_test.head()